In [1]:
import csv
import MySQLdb
import sys
import os
import genCSV as gc

In [2]:
# Class kernel
class kernel:

    def __init__(self, entry):
        self.entry = entry

    def get_cid(self):
        return str(self.entry[0])

    def get_entry(self):
        return self.entry

    def kernel2csv(self):

        final = list([self.entry[0]] + [str(self.entry[1])] + [str(self.entry[2])] + [self.entry[7]])
        compressed = self.entry[8].split(" , ")
        parse_array = [i.split(" : ") for i in compressed]
        cprss = [fin[1] for fin in parse_array]
        final += cprss

        return final

In [3]:
# Virtual machine IP adress
HOST            = "148.60.11.195"

## MySQL user
DB_USER         = "script2"

## MySQL password
DB_PASSWD       = "ud6cw3xNRKnrOz6H"

In [4]:
# Create a new kernel instance from the physical kernel
def compute_kernel(cid):
    kernels_array = []
    
    try:
        socket = MySQLdb.connect(HOST, DB_USER, DB_PASSWD, "IrmaDB_prod")
        cursor = socket.cursor()
        
        for c in cid:
            query = "SELECT * FROM Compilations WHERE cid = " + str(c)
            cursor.execute(query)
            entry = cursor.fetchone()
            
            kernels_array.append(kernel(entry))
    
    except Exception as e:
        print(str(e),"\n" + "Unable to connect to database c = " + str(c), file=sys.stderr)
        exit(-1)
        
    finally:
        cursor.close()
        socket.close()

    return kernels_array

In [5]:
def create_header():

    conn = MySQLdb.connect(HOST, DB_USER, DB_PASSWD, "IrmaDB_prod")
    cursor = conn.cursor()

    get_prop = "SELECT name, type FROM Properties"
    # Extract properties
    cursor.execute(get_prop)
    types_results = list(cursor.fetchall())

    if len(types_results) == 0:
        print("\nError : Properties not present in database - You need to run Kanalyser first (https://github.com/TuxML/Kanalyser)")

    cursor.close()
    conn.close()

    # .config column names
    names = [""]*len(types_results)
    index = 0
    for (name, typ) in types_results:
        names[index] = name
        index += 1

    with open("config_bdd.csv", "w") as f:
        head = ("cid,date,time,vmlinux,GZIP-bzImage,GZIP-vmlinux,GZIP,BZIP2-bzImage,BZIP2-vmlinux,BZIP2,LZMA-bzImage,LZMA-vmlinux,LZMA,XZ-bzImage,XZ-vmlinux,XZ,LZO-bzImage,LZO-vmlinux,LZO,LZ4-bzImage,LZ4-vmlinux,LZ4," + ",".join(names)).split(",")
        writer = csv.DictWriter(f, head)
        writer.writeheader()
print("config_bdd.csv created", flush=True)

config_bdd.csv created


In [6]:
def to_csv(From, To):    
    values = gc.genCSV(0, From, To)
    
    cid_array = [c for c in range(From, To)]
    kernels_array = compute_kernel(cid_array)
    
    with open("config_bdd.csv", "a") as csv_file:
        writer = csv.writer(csv_file)

        for num, k in enumerate(kernels_array):
            
            entry = k.kernel2csv() + values[num]       
            writer.writerow(entry)

    


In [7]:
if __name__ == "__main__":
    
    # Change values to append a new range of data from db to the csv
    From = 70000
    To = 70002
    
    if not os.path.exists("config_bdd.csv"):
        create_header()
    
    to_csv(From, To)
    print("\nWritten!")

Connecting to db IrmaDB_prod at 148.60.11.195...Done
Filling rows :
Progression: [--------------------] 0%####################] 100%
Written!
